# Задание 1

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings("ignore")
import os
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
import spacy
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import DistilBertTokenizerFast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/ My\ Drive/Colab\ Notebooks


/content/drive/ My Drive/Colab Notebooks


In [ ]:
class BbcDataset(object):
    def __init__(self):
        self.DATA_DIR = "bbc/"
        self.classes = os.listdir(self.DATA_DIR)
        self.n_documents = len([os.path.join(path, name) for path, subdirectory, files in os.walk(self.DATA_DIR) for name in files])

    def get_dataset(self):
        data = []
        for file in [os.path.join(path, name) for path, subdirectory, files in os.walk(self.DATA_DIR) for name in files]:
            label = file.split("/")[-2]
            content = " ".join(open(file, encoding='utf-8', errors='ignore').read().splitlines())
            data.append((content, label))
        return data

    def get_pandas_alike_dataset(self):
        data = self.get_dataset()
        return pd.DataFrame(data, columns=["text", "label"])

In [ ]:
df = BbcDataset().get_pandas_alike_dataset()
df = df[df.label != "bbc"]
df['label'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: label, dtype: int64

In [ ]:
class DataPreproces(object):
    def __init__(self, df, df_text):
        self.df = df
        self.df_text = df_text
        self.regex = re.compile("[A-Za-z]+")
        self.mystopwords = stopwords.words('english') 

    def words_only(self, text):
        return " ".join(self.regex.findall(text))

    def remove_stopwords(self, text):
        try:
            return " ".join([token for token in text.split() if not token in self.mystopwords])
        except:
            return ""

    def parse_filter_document(self, text):
        filtered_doc = []
        for token in text:
            if token.is_stop == False | token.is_punct == False | token.is_space == False:
                if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']:
                  filtered_doc.append(token.lemma_)

        return ' '.join(filtered_doc)
    
    def final_dataset(self):
        self.df_text = self.df_text.str.lower()
        self.df_text = self.df_text.apply(self.words_only)
        self.df_text = self.df_text.apply(self.remove_stopwords)
        
        nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])
        
        self.df['processed_text'] = self.df_text.apply(lambda x: self.parse_filter_document(nlp(x)))
        
        data = self.df[['processed_text','label']]

        enc = LabelEncoder()
        data['label'] = enc.fit_transform(data['label'])
        
        return data

In [ ]:
data = DataPreproces(df, df.text).final_dataset()
data

,processed_text,label
1,election deal falter heath role tory fail hold...,2
2,stress gap public trust handle economy restore...,2
3,stalemate pension strike talk talk aim avert n...,2
4,tory candidate quit remark conservative electi...,2
5,act detention rule urge government act quickly...,2
...,...,...
2221,stormy year property insurer stre storm typhoo...,0
2222,german growth go reverse economy shrink month ...,0
2223,oil company russian setback international oil ...,0
2224,boss payout director agree pay include pocket ...,0


In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(data['processed_text'],data['label'], test_size=0.2, shuffle=True)

X_train, X_val, y_train, y_val = train_test_split(data['processed_text'],data['label'], test_size=0.1, shuffle=True)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape)


(2002,) (2002,) (445,) (445,) (223,) (223,)


## Часть 1. Классические алгоритмы машинного обучения

In [ ]:
mystopwords = stopwords.words('english')

tfidf = TfidfVectorizer(stop_words=mystopwords)

X_train_vec = tfidf.fit_transform(X_train).toarray()

X_test_vec = tfidf.transform(X_test).toarray()

X_val_vec = tfidf.transform(X_val).toarray()


In [ ]:
def Model(models, X_train, X_test, X_val, y_train, y_test, y_val):    

    for key in models.keys():
    
        model = models[key]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)
        
        print(key)
        print("Train quality: \n")
        print("Accuracy: %1.3f \tPrecision: %1.3f \tRecall: %1.3f \t\tF1: %1.3f\n" % (accuracy_score(y_train, y_train_pred), 
                                                                                      precision_score(y_train, y_train_pred, average='macro'), 
                                                                                      recall_score(y_train, y_train_pred, average='macro'), 
                                                                                      f1_score(y_train, y_train_pred, average='macro')))
        print("Test quality: ")
        print("Accuracy: %1.3f \tPrecision: %1.3f \tRecall: %1.3f \t\tF1: %1.3f\n" % (accuracy_score(y_test, y_pred), 
                                                                                      precision_score(y_test, y_pred, average='macro'), 
                                                                                      recall_score(y_test, y_pred, average='macro'), 
                                                                                      f1_score(y_test, y_pred, average='macro')))
        print("Validation quality: ")
        print("Accuracy: %1.3f \tPrecision: %1.3f \tRecall: %1.3f \t\tF1: %1.3f\n" % (accuracy_score(y_val, y_val_pred), 
                                                                                      precision_score(y_val, y_val_pred, average='macro'), 
                                                                                      recall_score(y_val, y_val_pred, average='macro'), 
                                                                                      f1_score(y_val, y_val_pred, average='macro')))
        print("_______________________\n")
 
    return

models = {
    'DecisionTree': DecisionTreeClassifier(criterion='gini', max_depth=100, min_samples_split= 2),  
    'LogisticRegression': LogisticRegression(), 
    'RandomForestClassifier': RandomForestClassifier(criterion='entropy', max_depth= 50, n_estimators= 150)
}

Model(models, X_train_vec, X_test_vec, X_val_vec, y_train, y_test, y_val)

 

DecisionTree
Train quality: 

Accuracy: 1.000 	Precision: 1.000 	Recall: 1.000 		F1: 1.000

Test quality: 
Accuracy: 0.991 	Precision: 0.990 	Recall: 0.991 		F1: 0.991

Validation quality: 
Accuracy: 0.874 	Precision: 0.878 	Recall: 0.876 		F1: 0.877

_______________________

LogisticRegression
Train quality: 

Accuracy: 0.996 	Precision: 0.995 	Recall: 0.995 		F1: 0.995

Test quality: 
Accuracy: 1.000 	Precision: 1.000 	Recall: 1.000 		F1: 1.000

Validation quality: 
Accuracy: 0.987 	Precision: 0.986 	Recall: 0.984 		F1: 0.985

_______________________

RandomForestClassifier
Train quality: 

Accuracy: 1.000 	Precision: 1.000 	Recall: 1.000 		F1: 1.000

Test quality: 
Accuracy: 0.998 	Precision: 0.998 	Recall: 0.997 		F1: 0.997

Validation quality: 
Accuracy: 0.964 	Precision: 0.963 	Recall: 0.961 		F1: 0.962

_______________________



## Часть 2. DistilBert

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer.max_model_input_sizes

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropo

{'distilbert-base-uncased': 512,
 'distilbert-base-uncased-distilled-squad': 512,
 'distilbert-base-cased': 512,
 'distilbert-base-cased-distilled-squad': 512,
 'distilbert-base-german-cased': 512,
 'distilbert-base-multilingual-cased': 512}

In [ ]:
train_encodings = tokenizer(X_train.to_list(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.to_list(), truncation=True, padding=True)
val_encodings = tokenizer(X_val.to_list(), truncation=True, padding=True)


In [ ]:
class BBC_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = BBC_Dataset(train_encodings, y_train.to_list())
test_dataset = BBC_Dataset(test_encodings, y_test.to_list())
val_dataset = BBC_Dataset(val_encodings, y_val.to_list())


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5).to(device)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.22.2",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/mo

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }


training_args = TrainingArguments(
    output_dir='./outputs',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="steps",
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,                       
    args=training_args,     
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics= compute_metrics
)


using `logging_steps` to initialize `eval_steps` to 50
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [ ]:
trainer.train()
trainer.save_model("bbc_model")


***** Running training *****
  Num examples = 2002
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 753


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.906900,0.253116,0.943820,0.943132,0.950459,0.940757
100,0.199700,0.126680,0.968539,0.966871,0.967343,0.968280
150,0.189000,0.109248,0.973034,0.972814,0.972452,0.974404
200,0.184600,0.046176,0.986517,0.986080,0.987718,0.984884
250,0.126800,0.028813,0.988764,0.988453,0.989619,0.987551
300,0.154500,0.088469,0.975281,0.974687,0.974546,0.975835
350,0.050400,0.045521,0.988764,0.988762,0.990099,0.987922
400,0.044600,0.086281,0.982022,0.981773,0.981371,0.982737
450,0.071000,0.005330,1.000000,1.000000,1.000000,1.000000
500,0.030500,0.061490,0.984270,0.983405,0.984308,0.983246


***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 445
  Batch siz

In [ ]:
q=[trainer.evaluate(eval_dataset=data) for data in [train_dataset, test_dataset, val_dataset]]
pd.DataFrame(q, index=["train","test","val"]).iloc[:,:5]

***** Running Evaluation *****
  Num examples = 2002
  Batch size = 4


***** Running Evaluation *****
  Num examples = 445
  Batch size = 4
***** Running Evaluation *****
  Num examples = 223
  Batch size = 4


,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.016103,0.997502,0.997472,0.997474,0.997472
test,0.001355,1.000000,1.000000,1.000000,1.000000
val,0.065381,0.991031,0.990156,0.990040,0.990524


## Вывод

Подход DistilBert отрабатывает лучше классических методов машинного обучения, практически безошибочно предсказывая класс выборок test и validation. 